In [199]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [200]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [201]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 150
LOSS_FUNCTION = nn.MSELoss()
LOWEST_LOSS = 150000

In [202]:
y_loss = []
X_epochs = []

In [203]:
def data_workflow(input_data, target_value):
    X = input_data.drop([target_value], axis=1)
    y = input_data[target_value]

    #print(X)
    #print(y)

    X = X.to_numpy()
    y = y.to_numpy()

    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    data = TensorDataset(X, y)

    train_ds, test_ds = train_test_split(data, test_size=0.2, random_state=25)

    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

    print(f"training data: {train_dl}\n test data: {test_dl}")
    return train_dl, test_dl
    



In [204]:
csv_filepath = "./final_harvest_data.csv"

data = pd.read_csv(csv_filepath)
DATA = data.drop(['date', 'index', 'plant_id', 'tray_id', 'row', 'column'], axis=1)

DATA

,LFW_g,LDW_g,LA_mm2,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path,plant_convex_hull_vertices,plant_ellipse_major_axis,plant_ellipse_minor_axis,plant_ellipse_angle,plant_ellipse_eccentricity
0,1.30,0.078,31.95,4.3,5.2,5.8,1211,1456.0,0.831731,189.923880,49,42,346,16,45.684494,40.965988,92.878510,0.442608
1,2.10,0.148,44.10,5.3,5.7,5.5,1412,1556.5,0.907164,181.338094,56,41,370,20,53.368065,37.484673,96.255425,0.711802
2,3.36,0.196,67.61,7.3,6.5,8.9,1303,1766.5,0.737617,247.865005,49,54,333,19,46.356819,43.059174,2.102176,0.370421
3,3.07,0.184,66.98,5.8,7.6,7.5,1601,1787.0,0.895915,203.480229,44,59,395,21,55.633369,38.548523,15.127802,0.721031
4,3.17,0.187,68.74,7.2,8.0,7.1,1919,2372.0,0.809022,263.421354,62,58,454,19,60.012882,48.875011,55.774792,0.580292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,57.96,3.010,726.46,18.5,18.5,13.0,19084,22974.0,0.830678,767.938160,193,167,1367,33,190.580887,137.733093,62.138748,0.691160
161,81.46,3.880,1001.79,21.0,21.5,14.8,18373,21556.5,0.852318,661.612260,164,189,1317,29,176.040924,141.141724,22.056726,0.597653
162,140.84,6.380,1707.14,23.0,22.5,16.9,23374,26050.0,0.897274,683.754395,186,194,1427,31,191.379974,164.496170,175.959305,0.511091
163,108.17,5.250,1364.18,31.4,20.5,16.6,23457,25678.5,0.913488,671.754395,198,179,1364,31,194.815384,159.870819,122.656914,0.571464


In [205]:
LFW_train, LFW_test = data_workflow(DATA, 'LFW_g')

training data: <torch.utils.data.dataloader.DataLoader object at 0x7faafed172e0>
 test data: <torch.utils.data.dataloader.DataLoader object at 0x7faafed17610>


In [206]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(17, 16),
            #nn.ReLU(),
            nn.Linear(16, 1),
            
        ) 

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits.double()
        



In [207]:
model = DeepNeuralNetwork()
model = model.double()
print(model)

DeepNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [208]:
def train_loop(dataset, model, loss_function, optimizer, filler):
    model.train()
    for (X, y) in dataset:
        #X, y = X.to('cuda'), y.to('cuda')
        y = y.view(-1,1) 
        
        model.zero_grad()
        optimizer.zero_grad()

        prediction = model(X)
        loss = loss_function(prediction, y)
        

        loss.backward()
        optimizer.step()

        global LOWEST_LOSS
        
        print(f"Loss: {loss}")
        if (loss < LOWEST_LOSS):
            LOWEST_LOSS = loss
        #print(f"X: {X} \n Y: {y}")

In [209]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n------------------------------------")
    train_loop(LFW_train, model, LOSS_FUNCTION, optimizer, LOWEST_LOSS)
    #test_loop(LFW_test, model, LOSS_FUNCTION)
print("Training complete!")
print(LOWEST_LOSS)

Epoch 1
------------------------------------
Loss: 1385327.6674703239
Loss: 720981.4472286827
Loss: 518467.7713298213
Loss: 662418.5622494407
Loss: 366159.35884867405
Epoch 2
------------------------------------
Loss: 90130.26001977903
Loss: 51042.68273551899
Loss: 17005.960633434
Loss: 5697.364153416094
Loss: 8536.38425690664
Epoch 3
------------------------------------
Loss: 23065.16531508395
Loss: 52657.95272496808
Loss: 98541.73887730758
Loss: 149364.2619440767
Loss: 56667.281774710515
Epoch 4
------------------------------------
Loss: 160750.06584043216
Loss: 149736.64447509183
Loss: 87493.93567283501
Loss: 99964.26953277265
Loss: 29075.954927632574
Epoch 5
------------------------------------
Loss: 68198.22409979916
Loss: 37988.72854577445
Loss: 32187.73419636184
Loss: 9486.725563199716
Loss: 4084.1734165897124
Epoch 6
------------------------------------
Loss: 1551.7951002323282
Loss: 252.33441628959753
Loss: 2094.1716617025486
Loss: 13646.912143786562
Loss: 14774.209408459203
E